In [0]:
#importando as bibliotecas
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import time
import numpy as np

In [0]:
#importando os dados
data_train = pd.read_csv('train.csv')

In [3]:
#visualizando os dados
data_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [4]:
#tornando a identificação do passageiro como indice(usarei isso mais tarde)
data_train.set_index('PassengerId', inplace = True)
data_train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [5]:
#verificando se há valores nulos
data_train.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [6]:
for i in [1,2,3]:
  count_NAN = data_train.query(f'Pclass == {i}')['Age'].isnull().sum()
  print(f' class = {i} has {count_NAN} values NAN')

 class = 1 has 30 values NAN
 class = 2 has 11 values NAN
 class = 3 has 136 values NAN


In [7]:
#explorando as idades e percebendo que a classe tem influência na média das idades.
data_train.groupby('Pclass')['Age'].mean()

Pclass
1    38.233441
2    29.877630
3    25.140620
Name: Age, dtype: float64

In [8]:
#explorando as idades e percebendo que o sexo também possui influência
data_train.groupby('Sex')['Age'].mean()

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

In [9]:
#verificando a media das idades das mulheres por classe
ages_women_classes_mean = data_train.query('Sex  == "female" ').groupby('Pclass')['Age'].mean()
ages_women_classes_mean

Pclass
1    34.611765
2    28.722973
3    21.750000
Name: Age, dtype: float64

In [10]:
#verificando a media das idades dos homens por classe, e percebe-se que o sexo também possui influência.
ages_men_classes_mean = data_train.query('Sex  == "male" ').groupby('Pclass')['Age'].mean()
ages_men_classes_mean

Pclass
1    41.281386
2    30.740707
3    26.507589
Name: Age, dtype: float64

In [11]:
#vetorizando os sexos
data_train['Sex'].replace(['male','female'],[0,1], inplace = True)
data_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


In [0]:
from sklearn.impute import KNNImputer
'''
substituindo os valores nulos em base nas suas características, 
por ex: se for homem e estava na 1º classe, os nulos serão substituidos em base nestes.
'''
fillna_knn = KNNImputer(n_neighbors = 5, weights="uniform",copy=False)

ages_class1_man_with_values_NAN = data_train.query('Pclass == 1 and Sex == 0')[['Age']]
ages_class1_womam_with_values_NAN = data_train.query('Pclass == 1 and Sex == 1')[['Age']]

ages_class1_man = pd.Series(fillna_knn.fit_transform(ages_class1_man_with_values_NAN).round().flatten())
ages_class1_womam = pd.Series(fillna_knn.fit_transform(ages_class1_womam_with_values_NAN).round().flatten())

ages_class2_man_with_values_NAN = data_train.query('Pclass == 2 and Sex == 0')[['Age']]
ages_class2_womam_with_values_NAN = data_train.query('Pclass == 2 and Sex == 1')[['Age']]

ages_class2_man= pd.Series(fillna_knn.fit_transform(ages_class2_man_with_values_NAN).round().flatten())
ages_class2_womam = pd.Series(fillna_knn.fit_transform(ages_class2_womam_with_values_NAN).round().flatten())

ages_class3_man_with_values_NAN = data_train.query('Pclass == 3 and Sex == 0')[['Age']]
ages_class3_womam_with_values_NAN = data_train.query('Pclass == 3 and Sex == 1')[['Age']]

ages_class3_man = pd.Series(fillna_knn.fit_transform(ages_class3_man_with_values_NAN).round().flatten())
ages_class3_womam = pd.Series(fillna_knn.fit_transform(ages_class3_womam_with_values_NAN).round().flatten())

ages_class1_man.index = ages_class1_man_with_values_NAN.index
ages_class1_womam.index = ages_class1_womam_with_values_NAN.index

ages_class2_man.index = ages_class2_man_with_values_NAN.index
ages_class2_womam.index = ages_class2_womam_with_values_NAN.index

ages_class3_man.index = ages_class3_man_with_values_NAN.index
ages_class3_womam.index = ages_class3_womam_with_values_NAN.index

In [13]:
#criando os novos dados para a tabela
new_ages = pd.concat([ages_class1_man,
                          ages_class1_womam,
                          ages_class2_man,
                          ages_class2_womam,
                          ages_class3_man,
                          ages_class3_womam])
len(new_ages)

891

In [14]:
#adicionando a nova coluna a base de dados.l
data_train['new_ages'] = new_ages
data_train

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,new_ages
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S,22.0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C,38.0
3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,26.0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S,35.0
5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",0,27.0,0,0,211536,13.0000,NaN,S,27.0
888,1,1,"Graham, Miss. Margaret Edith",1,19.0,0,0,112053,30.0000,B42,S,19.0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,NaN,1,2,W./C. 6607,23.4500,NaN,S,22.0


In [0]:
# selecionando os parametros de treino e as classes para treino
x = data_train[['new_ages','Pclass','Sex','SibSp','Parch']]
scaler = StandardScaler()
x = scaler.fit_transform(x)
y = data_train['Survived']

In [16]:
# procurando os melhores parâmetros para o modelo
SEED = 42
np.random.seed(SEED)
params = {
  'n_neighbors' : range(3,25),
  'leaf_size' : range(30,41),
  'weights': ['uniform','distance'],
  'algorithm' : ['ball_tree','kd_tree','brute'],
  'p' : range(1,4)
}
tic = time.time()
search =  GridSearchCV(KNeighborsClassifier(),
                   params,
                   cv = StratifiedKFold(n_splits=10))

search.fit(x,y)
tac = time.time()
delta_t = tac - tic
print('time:%.2f seconds'% delta_t)
results = pd.DataFrame(search.cv_results_)
results_sorted = results.sort_values('mean_test_score', ascending= False)
results_sorted.head()

time:478.02 seconds


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
3186,0.000826,0.000022,0.004457,0.000079,brute,32,6,1,uniform,"{'algorithm': 'brute', 'leaf_size': 32, 'n_nei...",0.755556,0.831461,0.752809,0.865169,0.865169,0.842697,0.831461,0.808989,0.88764,0.831461,0.827241,0.042225,1
2922,0.000839,0.000183,0.004746,0.000785,brute,30,6,1,uniform,"{'algorithm': 'brute', 'leaf_size': 30, 'n_nei...",0.755556,0.831461,0.752809,0.865169,0.865169,0.842697,0.831461,0.808989,0.88764,0.831461,0.827241,0.042225,1
3846,0.000739,0.000043,0.004282,0.000104,brute,37,6,1,uniform,"{'algorithm': 'brute', 'leaf_size': 37, 'n_nei...",0.755556,0.831461,0.752809,0.865169,0.865169,0.842697,0.831461,0.808989,0.88764,0.831461,0.827241,0.042225,1
3714,0.000820,0.000029,0.004448,0.000086,brute,36,6,1,uniform,"{'algorithm': 'brute', 'leaf_size': 36, 'n_nei...",0.755556,0.831461,0.752809,0.865169,0.865169,0.842697,0.831461,0.808989,0.88764,0.831461,0.827241,0.042225,1
3582,0.000795,0.000043,0.004429,0.000098,brute,35,6,1,uniform,"{'algorithm': 'brute', 'leaf_size': 35, 'n_nei...",0.755556,0.831461,0.752809,0.865169,0.865169,0.842697,0.831461,0.808989,0.88764,0.831461,0.827241,0.042225,1


In [19]:
for index, row in results_sorted.head(1).iterrows():
  print("confidence_interval = [%.3f +- %.3f]" % (row.mean_test_score - row.std_test_score * 2,
                                                  row.mean_test_score + row.std_test_score * 2))

confidence_interval = [0.743 +- 0.912]


In [0]:
model = search.best_estimator_

# Testando o modelo

In [21]:
data_test = pd.read_csv('test.csv')
data_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [22]:
data_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [23]:
for i in [1,2,3]:
  count_NAN = data_test.query(f'Pclass == {i}')['Age'].isnull().sum()
  print(f' class = {i} has {count_NAN} values NAN')

 class = 1 has 9 values NAN
 class = 2 has 5 values NAN
 class = 3 has 72 values NAN


In [24]:
data_test['Sex'].replace(['male','female'],[0,1], inplace = True)
data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,S


In [0]:
'''
substituindo os valores nulos em base nas suas características, 
por ex: se for homem e estava na 1º classe, os nulos serão substituidos em base nestes.
'''
fillna_knn = KNNImputer(n_neighbors = 5, weights="uniform",copy=False)

ages_class1_man_with_values_NAN = data_test.query('Pclass == 1 and Sex == 0')[['Age']]
ages_class1_womam_with_values_NAN = data_test.query('Pclass == 1 and Sex == 1')[['Age']]

ages_class1_man = pd.Series(fillna_knn.fit_transform(ages_class1_man_with_values_NAN).round().flatten())
ages_class1_womam = pd.Series(fillna_knn.fit_transform(ages_class1_womam_with_values_NAN).round().flatten())

ages_class2_man_with_values_NAN = data_test.query('Pclass == 2 and Sex == 0')[['Age']]
ages_class2_womam_with_values_NAN = data_test.query('Pclass == 2 and Sex == 1')[['Age']]

ages_class2_man= pd.Series(fillna_knn.fit_transform(ages_class2_man_with_values_NAN).round().flatten())
ages_class2_womam = pd.Series(fillna_knn.fit_transform(ages_class2_womam_with_values_NAN).round().flatten())

ages_class3_man_with_values_NAN = data_test.query('Pclass == 3 and Sex == 0')[['Age']]
ages_class3_womam_with_values_NAN = data_test.query('Pclass == 3 and Sex == 1')[['Age']]

ages_class3_man = pd.Series(fillna_knn.fit_transform(ages_class3_man_with_values_NAN).round().flatten())
ages_class3_womam = pd.Series(fillna_knn.fit_transform(ages_class3_womam_with_values_NAN).round().flatten())

ages_class1_man.index = ages_class1_man_with_values_NAN.index
ages_class1_womam.index = ages_class1_womam_with_values_NAN.index

ages_class2_man.index = ages_class2_man_with_values_NAN.index
ages_class2_womam.index = ages_class2_womam_with_values_NAN.index

ages_class3_man.index = ages_class3_man_with_values_NAN.index
ages_class3_womam.index = ages_class3_womam_with_values_NAN.index

In [26]:
new_ages = pd.concat([ages_class1_man,
                          ages_class1_womam,
                          ages_class2_man,
                          ages_class2_womam,
                          ages_class3_man,
                          ages_class3_womam])
data_test['new_ages'] = new_ages
data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,new_ages
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,Q,34.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,S,47.0
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,Q,62.0
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,S,27.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,S,22.0


In [27]:
data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,new_ages
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,Q,34.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,S,47.0
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,Q,62.0
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,S,27.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,S,22.0


In [0]:
x_test = data_test[['new_ages','Pclass','Sex','SibSp','Parch']]
scaler = StandardScaler()
x_test = scaler.fit_transform(x_test)

In [0]:
y = model.predict(x_test)

In [32]:
# vendo como deve ser a submissão
example_of_submission = pd.read_csv('gender_submission.csv')
example_of_submission

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [0]:
# criando a tabela para a submissão
my_submission = pd.DataFrame(data_test['PassengerId'])
my_submission['Survived'] = y

In [0]:
my_submission.to_csv('my_submission.csv', index = False)

In [35]:
test1 = pd.read_csv('my_submission.csv')
test1.head()
# O score foi 0.75 segundo o algoritmo do keagle.

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
